In [5]:
%load_ext autoreload
%load_ext tensorboard
%autoreload 2
import os
import sys
sys.path.insert(0, os.path.abspath('../..'))
sys.path.insert(0, os.path.abspath('../../..'))

import importlib

import contextlib

import json
import os
import random

import numpy as np
import torch
from collections import defaultdict
from utils import get_default_path

import time
from utils import Stopwatch
import datetime

import ai.stabledisco as sd
import ai.torchmodules as torchmodules
import ai.torchmodules.data as torchdata
import ai.torchmodules.utils as torchutils
import ai.stabledisco.utils as sdutils
import clip

import torch
import torch.nn as nn
import pandas as pd
from collections import defaultdict

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [6]:
df_file_path = get_default_path("large_datasets", "aug_prompts.feather")
if "prompt_dataframe" in dir():
    del prompt_dataframe
prompt_dataframe = pd.read_feather(df_file_path)

In [24]:
df_file_path = get_default_path("large_datasets", "aug_prompts.feather")
prompt_dataframe.to_feather(df_file_path)

In [25]:
print(len(prompt_dataframe))

16798395


In [ ]:
import markovify
sentences = []

prompt_sample_num = 2000000

to_gen = 2000000
per_markov = to_gen//10
iters = to_gen // per_markov
for markov_num in range(iters):
    prompts = list(prompt_dataframe.sample(n=prompt_sample_num)["prompt"])
    print(f"Making markov {markov_num}")
    prompt_markov = markovify.NewlineText(prompts, 2, retain_original=False)
    del prompts
    prompt_markov.compile(inplace=True)
    for idx in range(per_markov):
        if idx % 2500 == 0:
            print(idx)
        sent = prompt_markov.make_sentence(test_output=False)
        if sent:
            sentences.append(sent)
    del prompt_markov

In [72]:
rows = []
for sent in sentences:
    if not sent:
        continue
    rows.append({
        "prompt": sent.lower(),
        "text_tokens": clip.tokenize(sent, truncate=True)[0].numpy().astype(np.uint16)  
    })
    
prompt_dataframe = pd.concat([prompt_dataframe, pd.DataFrame.from_records(rows)], ignore_index=True)
prompt_dataframe.drop_duplicates(subset="prompt", ignore_index=True, inplace=True)
prompt_dataframe = prompt_dataframe.sample(frac=1).reset_index(drop=True)

df_file_path = get_default_path("large_datasets", "aug_prompts_balanced.feather")
prompt_dataframe.to_feather(df_file_path)
print(len(prompt_dataframe))

18431569


In [ ]:
df_file_path = get_default_path("large_datasets", "aug_prompts_balanced.feather")

In [23]:
for row in prompt_dataframe.itertuples(True):
    idx = row[0]
    if type(row.text_tokens[0]) == np.ndarray:
        prompt_dataframe.at[idx, "text_tokens"] = row.text_tokens[0]
        

In [ ]:
random.shuffle(sentences)
print("\n\n".join(sentences[:1000]))

In [18]:
import pickle
df_file_path = get_default_path("large_datasets", "aug_prompts.feather")
markov_file_path = get_default_path("large_datasets", "aug_markov.pk")
with open(markov_file_path, 'wb+') as outfile:
    pickle.dump(prompt_markov, outfile)
    

In [7]:
import clip
from open_clip.tokenizer import _tokenizer as clip_tokenizer
eot_token = clip_tokenizer.encoder["<end_of_text>"]

vit14_clip_model, vit14_clip_preprocess = clip.load('ViT-L/14')

In [8]:
sot_token = clip_tokenizer.encoder["<start_of_text>"]

In [66]:
token_cnt = {token: 0 for token in clip_tokenizer.encoder.values()}
eos_token = clip
for row in prompt_dataframe.itertuples():
    for val in row.text_tokens:
        token_cnt[val] += 1
        if val == eot_token:
            break
            
    if row[0] % 500000 == 0:
        print(row[0])

199999
199999
199999
199999
199999
199999
199999
199999
199999
199999
199999
199999
199999
199999
199999
199999
199999
199999
199999
199999
199999
199999
199999
199999
199999
199999


KeyboardInterrupt: 

In [67]:
import numpy as np

cnt_token = [(val, key) for key, val in token_cnt.items()]
cnts = [pair[0] for pair in cnt_token]
cnt_token.sort()

target_min = np.percentile(cnts, 80)
replacement_min = np.percentile(cnts, 80)
replacement_max = np.percentile(cnts, 99.9)

for perc in range(0, 101, 5):
    print(f"perc: {perc}", np.percentile(cnts, perc))

to_add_cnts = [[target_min - val, key] for key, val in token_cnt.items() if val < target_min]
replacement_candidates = {key  for key, val in token_cnt.items() if replacement_max > val > replacement_min}

perc: 0 0.0
perc: 5 6.0
perc: 10 24.0
perc: 15 49.0
perc: 20 79.0
perc: 25 118.0
perc: 30 166.0
perc: 35 224.0
perc: 40 297.0
perc: 45 386.15000000000146
perc: 50 500.0
perc: 55 647.0
perc: 60 847.0
perc: 65 1133.550000000003
perc: 70 1523.0
perc: 75 2127.0
perc: 80 3099.600000000006
perc: 85 4345.949999999997
perc: 90 6858.900000000009
perc: 95 15370.199999999953
perc: 100 24530415.0


In [70]:
thresh = np.percentile(cnts, 70)
rare_tokens = {key for key, val in token_cnt.items() if val <  thresh}

In [ ]:
print(len(rare_tokens))

34777


9153608


In [ ]:
candidate_prompts = []
new_rows = []
eos_token = clip
keep_common_chance = 1.0
for row in prompt_dataframe.itertuples():
    for val in row.text_tokens:
        if val in rare_tokens:
            candidate_prompts.append(row.prompt)
            new_rows.append({
                "prompt": row.prompt,
                "text_tokens": row.text_tokens,
            })
            break
        if val == eot_token:
            if random.random() < keep_common_chance:
                new_rows.append({
                    "prompt": row.prompt,
                    "text_tokens": row.text_tokens,
                })
                
            break
            
    if row[0] % 500000 == 0:
        print(row[0])
        print(len(candidate_prompts))
        print(candidate_prompts[-1])
        print(len(new_rows))
        print()

In [ ]:
prompt_dataframe = prompt_dataframe(new_rows)
prompt_dataframe.reset_index(drop=True, inplace=True)
prompt_dataframe.drop_duplicates(subset="prompt", ignore_index=True, inplace=True)
prompt_dataframe = prompt_dataframe.sample(frac=1).reset_index(drop=True)
prompt_dataframe.to_feather(df_file_path)

In [71]:
import markovify
sentences = []

prompt_sample_num = 2000000

to_gen = 4000000
per_markov = to_gen//10
iters = to_gen // per_markov
for markov_num in range(iters):
    random.shuffle(candidate_prompts)
    prompts = candidate_prompts[:prompt_sample_num]
    print(f"Making markov {markov_num}")
    prompt_markov = markovify.NewlineText(prompts, 2, retain_original=False)
    del prompts
    prompt_markov.compile(inplace=True)
    for idx in range(per_markov):
        if idx % 2500 == 0:
            if "sent" in dir():
                print(sent)
        sent = prompt_markov.make_sentence(test_output=False)
        if sent:
            sentences.append(sent)
    del prompt_markov

Making markov 0
both the guy dressed in a fashionable white suit, detailed face, delicate features, teenage amelie poulain, fantasy, intricate, elegant, sharp focus, a digital illustration by ruan jia piotr jabłonski zaha hadid, stainless steel, titanuim mirror and chrome finish, iridescent bismuth surfaces, cyberpunk 2 0 2 2 
brough superior
husqvarna motorcycles
pietro perugino. the delivery of food & wine bar cabinet. best 25 haircuts with beards masculine haircut ideas on pinterest sparkle cleaners
indoor heated pool, hot tub, friendship, fun, peyote colors, ornate, digital art, inspired by seasonal lemon-flavored oreos. they are about the success of gone fishing personalised birthday cards cominlily jamesorg
queen nefertiti, highly detailed, 5 k extremely detailed oil painting portrait, colorful, octane render, splatter paint, desaturated rainbow color scheme, sense of awe, high fashion warrior, intricate geometric decoration in black/red
detailed art on artstation, ultra realisti

In [ ]:
idx = 0
for cnt, tok in to_add_cnts:
    if cnt < 1000:
        print(clip_tokenizer.decoder[tok])
    idx += 1

In [ ]:
import random
import copy
new_tokens = []

keep_chance = 0.75
for idx, orig_tokens in enumerate(prompt_dataframe["text_tokens"]):
    if idx % 100000 == 0:
        print(idx, "rem", len(to_add_cnts))
    replaced = 0
    tokens = copy.copy(orig_tokens)
    for idx, token in enumerate(tokens[1:]):
        if token == eot_token:
            break
            
        if token not in replacement_candidates or random.random() < keep_chance:
            continue
            
        replaced += 1
        rep_token_idx = random.randint(0, len(to_add_cnts)-1)
        rep_token = to_add_cnts[rep_token_idx][1]
        tokens[idx+1] = rep_token
        
        to_add_cnts[rep_token_idx][0] -= 1
        if to_add_cnts[rep_token_idx][0] < 1:
            del to_add_cnts[rep_token_idx]
            
        if len(to_add_cnts) < 1:
            break

    if replaced > 0:
        new_tokens.append(tokens)

    if len(to_add_cnts) < 1:
        break
print(len(new_tokens))
new_tokens = np.array(new_tokens)

In [ ]:
text = [sdutils.decode_clip_tokens(aug_tokens)[0] for aug_tokens in new_tokens]

In [ ]:
new_df = pd.DataFrame.from_dict({"prompt": text, "text_tokens": new_tokens.tolist()})
prompt_dataframe = pd.concat([prompt_dataframe, new_df], ignore_index=True) 
del new_df

In [84]:
prompt_dataframe = prompt_dataframe.sample(frac=1).reset_index(drop=True)
prompt_dataframe.to_feather(df_file_path)